<a href="https://colab.research.google.com/github/Hyun-Jun-Lee/Facial_Dection_PJ/blob/master/Gender_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d cashutosh/gender-classification-dataset

In [ ]:
!ls

In [ ]:
!unzip gender-classification-dataset.zip

In [ ]:
ls

In [ ]:
# 라이브러리 정리
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense , Activation , Dropout ,Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.metrics import categorical_accuracy
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint
from keras.optimizers import *
from keras.layers.normalization import BatchNormalization
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.callbacks import EarlyStopping
from google.colab import files
from skimage import io
import matplotlib.image as mpimg

In [ ]:
# 경로설정 
base_dir = "/content/"
train_dir = "/content/Training"
validation_dir = "/content/Validation"

train_male_dir = os.path.join(train_dir, "male")
train_female_dir = os.path.join(train_dir, "female")

valid_male_dir = os.path.join(validation_dir, "male")
valid_female_dir = os.path.join(validation_dir, "female")

In [ ]:
train_male_fnames = os.listdir(train_male_dir)
train_female_fnames = os.listdir(train_female_dir)

valid_male_fnames = os.listdir(valid_male_dir)
valid_female_fnames = os.listdir(valid_female_dir)

print(f"total training of male is : {len(train_male_fnames)}")
print(f"total training of female is : {len(train_female_fnames)}")
print(f"total validation of male is : {len(valid_male_fnames)}")
print(f"total validation of female is : {len(valid_female_fnames)}")

In [ ]:
# 데이터 확인

nrows = 4
ncols = 4

pic_index = 0

fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)

pic_index += 8
male_img = [os.path.join(train_male_dir, fname) for fname in train_male_fnames[pic_index-8 : pic_index]]
female_img = [os.path.join(train_female_dir, fname) for fname in train_female_fnames[pic_index-8 : pic_index]]


for i, img in enumerate(male_img + female_img):
    set_subplot = plt.subplot(nrows, ncols, i + 1)
    set_subplot.axis("Off")
    image = mpimg.imread(img)
    plt.imshow(image)
plt.show()

In [ ]:
def gendermodel():
    model = Sequential()
    input_shape=(300, 300, 3)

    model.add(Conv2D(16, (3,3), activation="relu", input_shape=input_shape))
    model.add(MaxPooling2D(2,2))

    model.add(Conv2D(32, (3,3), activation="relu"))
    model.add(MaxPooling2D(2,2))

    model.add(Conv2D(64, (3,3), activation="relu"))
    model.add(MaxPooling2D(2,2))
    
    model.add(Conv2D(64, (3,3), activation="relu"))
    model.add(MaxPooling2D(2,2))

    model.add(Flatten())
    model.add(Dense(512, activation="relu"))
    model.add(Dense(84, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))

    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

    return model
    
model = gendermodel()
model.summary()


In [ ]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])


In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255)
test_datagen = ImageDataGenerator(rescale=1/255)

training_generator = train_datagen.flow_from_directory(train_dir,
                                                      target_size=(300, 300),
                                                      batch_size=20,
                                                      class_mode="binary")

validation_generator = test_datagen.flow_from_directory(validation_dir,
                                                       target_size=(300, 300),
                                                       batch_size=20,
                                                       class_mode="binary")

In [ ]:
history = model.fit_generator(training_generator, steps_per_epoch=100, epochs=30,
                    validation_data=validation_generator,
                    validation_steps=50,
                    verbose=1)

In [ ]:
model.evaluate(training_generator)

In [ ]:
uploaded = files.upload()

In [ ]:
def my_model(path):
    img = image.load_img(path, target_size=(300, 300))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

    images_show = np.vstack([x])
    predict_classes = model.predict(images_show, batch_size=10)
    if predict_classes > 0.5:
        print(path + " Female")
    else:
        print(path + " Male")

    show_img = mpimg.imread(path)
    plt.imshow(show_img)
    plt.show()

In [ ]:
path = '/content/testing.jpg'

my_model(path)

In [ ]:
path2 = '/content/BIBI.jfif'

my_model(path2)